<a href="https://colab.research.google.com/github/anuragdhirubhai/DL_NLP_project/blob/main/sentiment_analysis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# we will be using BERT for sentiment analysis
# we will Transformaers package to get it done.

So, what we are going to do is we will install Transformers, use BERT for our sentiment scoring and scrape data of revew from yelp and score

**Installing Dependencies** 

In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

**Importing Libraries**

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

**2. Instantiate Model**

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

**3. Encode and Calculate Sentiment**

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [7]:
result = model(tokens)

In [8]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits))+1

4

4. Collect Reviews

In [20]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [21]:
reviews

["Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent.",
 "Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent.",
 'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 "It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places.",
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked s

**5. Load Reviews into DataFrame and Score**

In [22]:
import numpy as np
import pandas as pd

In [23]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [24]:
df['review'].iloc[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [25]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [26]:
sentiment_score(df['review'].iloc[1])

4

In [27]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [28]:
df

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,Great food amazing coffee and tea. Short walk ...,5
3,It was ok. Had coffee with my friends. I'm new...,3
4,Ricotta hot cakes! These were so yummy. I ate ...,5
5,Great staff and food. Must try is the pan fri...,5
6,I came to Social brew cafe for brunch while ex...,5
7,We came for brunch twice in our week-long visi...,4
8,It was ok. The coffee wasn't the best but it w...,3
9,I went here a little while ago- a beautiful mo...,2


In [29]:
df['review'].iloc[3]

"It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places."